In [1]:
from tqdm import tqdm
import json

In [2]:
from glob import glob

parasci = sorted(glob('parasci-*.json'))
parasci = [f for f in parasci if 'parasci-arxiv-test' not in f]
files = ['paws-train.json', 'paws-dev.json', 'paws-test.json', 'mrpc-train.json'] + parasci
files

['paws-train.json',
 'paws-dev.json',
 'paws-test.json',
 'mrpc-train.json',
 'parasci-arxiv-train.json',
 'parasci-arxiv-val.json',
 'parasci-test.json',
 'parasci-train.json',
 'parasci-val.json']

In [3]:
funpedia = glob('funpedia*')
funpedia

['funpedia-train.json', 'funpedia-test.json', 'funpedia-valid.json']

In [4]:
with open(funpedia[0]) as fopen:
    data = json.load(fopen)

In [5]:
rejected = {'i', 'you', 'she', 'her', 'he', 'them'}

selected = []
for d in data:
    if len(set(d['label'].lower().split()) & rejected):
        continue
        
    if len(set(d['passage'].lower().split()) & rejected):
        continue
    
    selected.append(d)
    
len(data), len(selected)

(81467, 51585)

In [6]:
selected[:10]

[{'title': 'Soldier Mountain',
  'label': 'What a nice place to go above sea level, Soldier Mountain is',
  'passage': 'Located north of Fairfield in very rural Camas County, its summit elevation is above sea level with a vertical drop of .',
  'persona': 'Caring',
  'ms': ['Tempat yang bagus untuk naik ke atas permukaan laut, Soldier Mountain adalah',
   'Terletak di utara Fairfield di Camas County yang sangat luar bandar, ketinggian puncaknya berada di atas paras laut dengan penurunan menegak.']},
 {'title': 'Tyssøy',
  'label': 'Incredible, the island Tyssoy is all away towards the north in the country',
  'passage': 'The island lies in the northern part of the Raunefjorden.',
  'persona': 'Profound',
  'ms': ['Luar biasa, pulau Tyssoy terletak di utara di negara ini',
   'Pulau ini terletak di bahagian utara Raunefjorden.']},
 {'title': 'Byfjorden (Rogaland)',
  'label': 'It would be so liberating to live in a fjord like Byfjorden in Norway.',
  'passage': 'Byfjorden is a fjord in 

In [7]:
with open('train.json', 'w') as fopen_jsonl:
    
    for f in files:
        with open(f) as fopen:
            data = json.load(fopen)
    
        for i in tqdm(range(len(data))):
            ms = data[i]['ms']
            if len(ms) != 2:
                continue

            left = ms[0]
            right = ms[1]

            if left == right:
                continue

            if len(left) and len(right) and len(left.split()) < 256 and len(right.split()) < 256:
                d = {"translation": {"src": left, "tgt": right, 'prefix': 'parafrasa: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                d = {"translation": {"src": right, "tgt": left, 'prefix': 'parafrasa: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

100%|███████████████████████████████████| 2753/2753 [00:00<00:00, 177189.67it/s]


In [8]:
!wc -l train.json

745890 train.json


In [9]:
with open('test.json', 'w') as fopen_jsonl:
    
    files = ['mrpc-val.json', 'parasci-arxiv-test.json']
    for f in files:
        with open(f) as fopen:
            data = json.load(fopen)
        for i in tqdm(range(len(data))):
            ms = data[i]['ms']
            if len(ms) != 2:
                continue

            left = ms[0]
            right = ms[1]

            if left == right:
                continue

            if len(left) and len(right) and len(left.split()) < 256 and len(right.split()) < 256:
                d = {"translation": {"src": left, "tgt": right, 'prefix': 'parafrasa: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                d = {"translation": {"src": right, "tgt": left, 'prefix': 'parafrasa: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

100%|███████████████████████████████████| 2549/2549 [00:00<00:00, 174269.85it/s]


In [10]:
!wc -l test.json

5544 test.json


In [1]:
!shuf train.json > shuffled-train.json
!shuf test.json > shuffled-test.json